# Euchre
Goal: code up a playable euchre game to better understand winning probabilities of different starting hands  
Given: Right, Ace, Queen, off Ace, off ten; 3-suited, dealer spot - what's the likelihood of winning each number of tricks that round?  
Genetic Algorithm to determine next play would be cool - start with random strategy, adapt to new weights as games progress  
Streamlit app where user can select cards in hand and table position and get breakdown of likelihood of taking 5/4/3/2/1/0 tricks

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys

sys.path.insert(0, 'C:/Users/jerem/Desktop/nonsense/euchre/')

In [2]:
from utils import EuchreGame

In [3]:
euchre_game = EuchreGame()

In [4]:
euchre_game.score

{'t1': 0, 't2': 0}

In [5]:
euchre_game.dealer

'p1'

In [6]:
euchre_game.next_to_deal

['p2', 'p3', 'p4', 'p1']

In [7]:
euchre_game.print_score()

Current score: 0-0


In [8]:
euchre_game.card_suits

['S', 'C', 'H', 'D']

In [62]:
euchre_game.card_values

['9', 'T', 'J', 'Q', 'K', 'A']

In [10]:
deck_of_cards = euchre_game.shuffle_deck_of_cards()
for card in deck_of_cards:
    print(card, end=',')

9_S,9_C,9_H,9_D,T_S,T_C,T_H,T_D,J_S,J_C,J_H,J_D,Q_S,Q_C,Q_H,Q_D,K_S,K_C,K_H,K_D,A_S,A_C,A_H,A_D,

### Deal cards

In [11]:
player_hands, card_flipped_up = euchre_game.deal_hand()

In [12]:
player_hands

{'p1': ['Q_D', 'T_C', 'A_H', '9_D', 'A_S'],
 'p2': ['Q_C', 'Q_H', 'J_H', '9_S', 'J_S'],
 'p3': ['A_D', 'K_D', 'Q_S', 'T_D', 'J_C'],
 'p4': ['K_H', 'K_S', 'J_D', '9_H', 'T_S']}

In [13]:
card_flipped_up

'9_C'

### Choose suit for trump

In [14]:
if euchre_game.eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S']):
    print('order up trump')

In [15]:
if euchre_game.eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_S', 'J_C', 'J_S']):
    print('order up trump')

order up trump


In [16]:
euchre_game.choose_open_trump(hand=['K_S', 'A_S', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

'S'

In [17]:
euchre_game.choose_open_trump(hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

In [18]:
player_hands, card_flipped_up = euchre_game.deal_hand()
calling_player, trump = euchre_game.determine_trump(card_flipped_up=card_flipped_up,
                           player_hands=player_hands)

In [19]:
print(player_hands)
print(calling_player)
print(trump)

{'p1': ['K_H', 'Q_D', 'J_C', 'A_D', 'J_D'], 'p2': ['9_D', 'T_C', '9_C', 'J_H', 'A_C'], 'p3': ['K_C', '9_S', 'Q_S', 'T_D', 'J_S'], 'p4': ['A_S', 'Q_H', 'Q_C', 'A_H', '9_H']}
p3
S


In [20]:
euchre_game.play_card(hand=['J_H', 'A_D', '9_H', 'K_S', 'J_D'], 
          trump='S',
          cards_in_play=['K_H', 'T_H'], 
          suit_led='H')

'J_H'

In [181]:
def play_card(hand,
              trump,
              cards_in_play,
              suit_led=None):
    """
    Function to return card to play in hand
    TODO: check if partner has winning card_in_pot
    """
    card_values = {
        '9': 1,
        'T': 2,
        'J': 3,
        'Q': 4,
        'K': 5,
        'A': 6
    }
    # play last card
    if len(hand) == 1:
        return hand[0]
    
    # lead card
    if len(cards_in_play) < 1:
        for idx, card in enumerate(hand):
            # 1 - play right bauer
            if card[-1] == trump and card[0] == 'J':
                return hand[idx]
            # 2 - play off ace
            elif card[-1] != trump:
                if card[0] == 'A':
                    return hand[idx]
            # 3 - TODO: else play highest non-trump card
        return hand[0]
        
    # follow suit
    if suit_led is not None:
        # play lowest card in the suit played
        card_to_play_points = -1
        idx_to_return = -1
        for idx, card in enumerate(hand):
            if card[-1] == suit_led:
                card_points = card_values[card[0]]
                if card_points < card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        return hand[idx_to_return]
    
    # play other highest non-trump card
    else:
        card_to_play_points = -1
        idx_to_return = -1
        for idx, card in enumerate(hand):
            if card[-1] != trump:
                card_points = card_values[card[0]]
                if card_points > card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        if idx_to_return > -1:
            return hand[idx_to_return]
        
        else:  # only has trump left, play lowest trump
            trump_card_points = 9
            for idx, card in enumerate(hand):
                card_points = card_values[card[0]]
                if card_points < trump_card_points:
                    trump_card_points = card_points
                    idx_to_return = idx
            return hand[idx_to_return]


In [182]:
play_card(hand=['9_D', 'T_C', '9_C', 'J_H', 'A_C'],
          trump='S',
          cards_in_play=[],
          suit_led=None)

'A_C'

In [183]:
play_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'],
          trump='S',
          cards_in_play=[],
          suit_led=None)

'J_S'

In [184]:
play_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'],
          trump='S',
          cards_in_play=['K_C'],
          suit_led=None)

'A_C'

In [22]:
cards_in_play, player_led = euchre_game.play_trick(player_hands=player_hands, 
                                       trump=trump, 
                                       next_to_play_list=euchre_game.next_to_deal, 
                                       verbose=True)

['9_D', 'T_C', '9_C', 'J_H', 'A_C']
Player p2 plays 9_D
Suit led: D
['K_C', '9_S', 'Q_S', 'T_D', 'J_S']
Player p3 plays T_D
['A_S', 'Q_H', 'Q_C', 'A_H', '9_H']
Player p4 plays 9_H
['K_H', 'Q_D', 'J_C', 'A_D', 'J_D']
Player p1 plays A_D


In [23]:
cards_in_play

{'p2': '9_D', 'p3': 'T_D', 'p4': '9_H', 'p1': 'A_D'}

In [24]:
player_led

'p2'

In [40]:
player_hands, card_flipped_up = euchre_game.deal_hand()
# choose trump
calling_player, trump = euchre_game.determine_trump(card_flipped_up=card_flipped_up,
                                             player_hands=player_hands)

if trump is not None:
    trick_winners = {p: 0 for p in euchre_game.next_to_deal}
    next_to_play_list = euchre_game.next_to_deal
    for trick in range(5):
        cards_in_play, player_led = euchre_game.play_trick(player_hands=player_hands,
                                                    trump=trump,
                                                    next_to_play_list=next_to_play_list,
                                                    verbose=False)

D


In [41]:
player_led

'p2'

In [42]:
cards_in_play

{'p2': '9_S', 'p3': 'J_C', 'p4': 'Q_H', 'p1': 'T_H'}

In [53]:
'J_C' in cards_in_play.values()

True

In [69]:
cards_in_play

{'p2': '9_S', 'p3': 'J_C', 'p4': 'Q_H', 'p1': 'T_H'}

In [71]:
cards_in_play.values()

dict_values(['9_S', 'J_C', 'Q_H', 'T_H'])

In [58]:
trump_card = 'J_C'
player_with_highest_trump = [k for k, v in cards_in_play.items() if v == trump_card][0]

In [59]:
player_with_highest_trump

'p3'

In [43]:
trump

'D'

In [119]:
def determine_trick_winner(cards_in_play, 
                           trump, 
                           player_led, 
                           verbose=False) -> str:
    """
    Determine winner of trick
    
    Returns player that won trick
    """
    # create dict for each trump
    trump_hierarchy_dict = {
        'D':['J_D', 'J_H', 'A_D', 'K_D', 'Q_D', 'T_D', '9_D'],
        'H':['J_H', 'J_D', 'A_H', 'K_H', 'Q_H', 'T_H', '9_H'],
        'C':['J_C', 'J_S', 'A_C', 'K_C', 'Q_C', 'T_C', '9_C'],
        'S':['J_S', 'J_C', 'A_S', 'K_S', 'Q_S', 'T_S', '9_S']
    }
    # loop over trump cards, highest to lowest
    for trump_card in trump_hierarchy_dict[trump]:
        # if in cards_in_play:
        if trump_card in cards_in_play.values():
            if verbose:
                print(f'Trump card found: {trump_card}')
            # return player that played that card
            return [k for k, v in cards_in_play.items() if v == trump_card][0]
    led_suit = cards_in_play[player_led][-1]
    for card_val in euchre_game.card_values:
        for card_played in cards_in_play.values():
            if card_played[0]==card_val and card_played[-1]==led_suit:
                # return player that played that card
                return [k for k, v in cards_in_play.items() if v == card_played][0]

In [120]:
determine_trick_winner(cards_in_play={'p2': '9_S', 'p3': 'J_C', 'p4': 'J_H', 'p1': 'A_S'}, 
                           trump='S', 
                           player_led=player_led, verbose=True)

Trump card found: J_C


'p3'

In [121]:
determine_trick_winner(cards_in_play={'p2': '9_S', 'p3': 'J_C', 'p4': 'J_S', 'p1': 'A_S'}, 
                           trump='D', 
                           player_led=player_led)

Suit led:S
Card val: A,Card played: 9_S
Card played: J_C
Card played: J_S
Card played: A_S


'p1'

In [185]:
euchre_game = EuchreGame()

In [186]:
euchre_game.play_hand()

In [124]:
trick_winner = euchre_game.determine_trick_winner(cards_in_play=cards_in_play,
                                           trump=trump,
                                           player_led=player_led)

In [125]:
trick_winner

'p2'

In [128]:
euchre_game.play_full_game()

Current score: 0-1
Trump not found
Current score: 0-2
Current score: 2-2
Current score: 2-3
Current score: 3-3
Current score: 4-3
Current score: 5-3
Current score: 5-4
Current score: 7-4
Current score: 7-6
Current score: 7-7
Trump not found
Current score: 7-8
Current score: 7-9
Current score: 7-10
Current score: 7-10


In [187]:
%%time

euchre_game = EuchreGame()
euchre_game.play_full_game(verbose=False)

Wall time: 19.3 ms


In [188]:
cards_in_play

{'p2': '9_S', 'p3': 'J_C', 'p4': 'Q_H', 'p1': 'T_H'}

In [189]:
len(cards_in_play)

4

In [191]:
euchre_game.score['t2']

10

In [194]:
%%prun

euchre_game = EuchreGame()
euchre_game.play_full_game(verbose=False)

#### Loop through games played

In [192]:
%%time

games_played = 0
t1_wins = 0
for i in range(1000):
    euchre_game = EuchreGame()
    euchre_game.play_full_game(verbose=False)
    games_played += 1
    if euchre_game.score['t1'] > euchre_game.score['t2']:
        t1_wins += 1

print(games_played)
print(t1_wins)

1000
478
Wall time: 4.49 s


In [ ]:
# NEXT STEPS:
# build out play card strategy
    # add logic to check if cards in hand are better than those played
    # keep track of all cards played so far
        # check what trump has been played, etc.
# build out strategy on when to call trump
# track points for each hand type for each game
# add functionality to deal with dealer swapping out card_flipped_up
# add functionality to deal with loners
# add random strategy to play_card method
# add comments to all class methods
# check in all code to GitHub